# LLC Debuggin

In [1]:
# imports
# imports
from importlib import reload

import os
import numpy as np

import pandas
import xarray
import h5py

import torch
from tqdm.auto import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.models import io as model_io

In [2]:
sns.set_theme()
sns.set_style('whitegrid')
sns.set_context('notebook')

In [3]:
def pp_to_LL(pp_field, pae):
    # Latent
    pp_field.resize(1, 1, 64, 64)
    dset = torch.utils.data.TensorDataset(torch.from_numpy(pp_field).float())
    loader = torch.utils.data.DataLoader(
        dset, batch_size=1, shuffle=False,
        drop_last=False, num_workers=16)
    with torch.no_grad():
        latents = [pae.autoencoder.encode(data[0].to(pae.device)).detach().cpu().numpy()
                   for data in loader] #tqdm(loader, total=len(loader), unit='batch', desc='Computing latents')]
    print("Latents generated!")


    latents = pae.scaler.transform(np.concatenate(latents))
    #latents = np.concatenate(latents)

    # Debug
    #print("Debug: {}".format(np.sum(latents)))

    # LL
    dset = torch.utils.data.TensorDataset(torch.from_numpy(latents).float())
    loader = torch.utils.data.DataLoader(
        dset, batch_size=1024, shuffle=False,
        drop_last=False, num_workers=16)

    with torch.no_grad():
        log_prob = [pae.flow.log_prob(data[0].to(pae.device)).detach().cpu().numpy()
                        for data in loader] #tqdm(loader, total=len(loader), unit='batch', desc='Computing log probs')]
    print("Log probabilities generated!")
    
    # Return
    return log_prob[0]

# Load the main table

In [4]:
tbl_file = 's3://llc/Tables/test_uniform_r0.5_test.feather'
llc_table = ulmo_io.load_main_table(tbl_file)

Read main table: s3://llc/Tables/test_uniform_r0.5_test.feather


In [5]:
llc_table['DT'] = llc_table.T90 - llc_table.T10

# Load MODIS PAE (Ulmo)

In [6]:
pae = model_io.load_modis_l2(flavor='std', local=False)

Using device=cpu
Loading autoencoder model from: s3://modis-l2/Models/R2019_2010_128x128_std/autoencoder.pt
Loading flow model from: s3://modis-l2/Models/R2019_2010_128x128_std/flow.pt
scaler loaded from: s3://modis-l2/Models/R2019_2010_128x128_std/MODIS_R2019_2010_95clear_128x128_preproc_std_scaler.pkl


/home/xavier/Projects/anaconda3/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


# Bogus LL?

## High DT, LL=1000

In [7]:
high_DT = llc_table.DT > 4.

In [8]:
LL1000 = np.abs(llc_table.LL - 1000.) < 1
np.sum(LL1000)

800

In [9]:
llc_table[LL1000 & high_DT]

,lat,lon,row,col,datetime,LLC_file,pp_file,pp_root,field_size,mean_temperature,Tmin,Tmax,T90,T10,pp_idx,pp_type,LL,DT
103673,-42.213848,53.510418,5460,4360,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,7.573065,5.070,14.835000,11.124001,5.551000,39559,0,999.113403,5.573001
136246,50.477859,-43.718750,10931,16973,2011-11-30,s3://llc/ThetaUVSalt/LLC4320_2011-11-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,8.483759,5.957,10.844000,10.297000,6.287000,121046,0,999.287964,4.010000
145084,36.058659,148.364578,9903,8913,2011-11-30,s3://llc/ThetaUVSalt/LLC4320_2011-11-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,19.235039,16.417,22.192001,21.603001,16.598000,309044,0,1000.429565,5.005001
490085,-38.689117,22.843750,5700,2888,2012-03-31,s3://llc/ThetaUVSalt/LLC4320_2012-03-31T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,19.742813,17.528,23.162001,22.833000,18.447001,42448,0,999.895996,4.386000
494464,-42.213848,25.156250,5460,2999,2012-03-31,s3://llc/ThetaUVSalt/LLC4320_2012-03-31T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,13.304924,10.480,18.054001,15.654000,11.402000,118080,0,999.613159,4.252000
532255,40.678158,-71.197914,10209,15654,2012-05-31,s3://llc/ThetaUVSalt/LLC4320_2012-05-31T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,13.862737,10.584,17.153999,16.531000,11.399000,323579,0,1000.457397,5.132000
759293,-45.784721,62.156250,5204,4775,2012-07-31,s3://llc/ThetaUVSalt/LLC4320_2012-07-31T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,9.794881,7.307,13.709000,13.293000,7.630000,194881,0,1000.355652,5.663000


## Load PP img

In [10]:
cutout = llc_table.iloc[103673]
img, pp_hf = llc_io.grab_image(cutout, close=False)

In [11]:
cutout

lat                                                          -42.2138
lon                                                           53.5104
row                                                              5460
col                                                              4360
datetime                                          2011-09-30 00:00:00
LLC_file            s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...
pp_file                  s3://llc/PreProc/LLC_uniform_test_preproc.h5
pp_root                                                       llc_std
field_size                                                         64
mean_temperature                                              7.57307
Tmin                                                             5.07
Tmax                                                           14.835
T90                                                            11.124
T10                                                             5.551
pp_idx              

## Recalculate LL

In [12]:
LL = pp_to_LL(img, pae)

Latents generated!
Log probabilities generated!


In [14]:
#GPU : -45 or so [the number is not fixed]

In [13]:
LL

array([-46.58142], dtype=float32)